<a href="https://colab.research.google.com/github/vu-topics-in-big-data-2023/Team05/blob/main/census_tract_joins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#install spark. we are using the one that uses hadoop as the underlying scheduler.
!wget -q https://downloads.apache.org/spark/spark-3.2.4/spark-3.2.4-bin-hadoop3.2.tgz
!tar xf  spark-3.2.4-bin-hadoop3.2.tgz
!ls -l

#Provides findspark.init() to make pyspark importable as a regular library.
os.environ["SPARK_HOME"] = "spark-3.2.4-bin-hadoop3.2"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.7 pyspark-shell'

total 294144
drwx------  6 root root      4096 Apr 30 19:43 drive
drwxr-xr-x  1 root root      4096 Apr 27 13:35 sample_data
-rw-r--r--  1 root root      1266 Apr 30 19:42 schema.py
drwxr-xr-x 13 1000 1000      4096 Apr  9 21:17 spark-3.2.4-bin-hadoop3.2
-rw-r--r--  1 root root 301183180 Apr  9 21:35 spark-3.2.4-bin-hadoop3.2.tgz


In [ ]:
!pip install -q findspark pyspark
import findspark
findspark.init()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
!pip install geopandas 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 94.6 MB/s eta 0:00:00


In [ ]:
!pip install geospark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 3.8 MB/s eta 0:00:00


# Write Spark Code Locally and test the Code and Save it to your repository

# Step 2. Complete Spark Jobs Below Locally. 

Once they work you can submit them to EMR

In [3]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
file_path = "/content/drive/MyDrive/big-data-final/tennessee"

shapefile = gpd.read_file(file_path)

census_tracts = gpd.GeoDataFrame(shapefile)

points_df = pd.read_parquet('/content/drive/MyDrive/big-data-final/nfd_incidents_xd_seg.parquet')

points = gpd.GeoDataFrame(
    points_df, 
    crs=census_tracts.crs,  # Set the same CRS as the counties dataframe
    geometry=[Point(xy) for xy in zip(points_df.longitude, points_df.latitude)]
)

# Perform the spatial join
joined = gpd.sjoin(points, census_tracts, op='within')

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [4]:
# save the results of the spatial join to a parquet file
joined.to_parquet('/content/drive/MyDrive/big-data-final/tracts_joined_with_points.parquet')

In [6]:
census_info = pd.read_csv('/content/drive/MyDrive/big-data-final/census_economic_data_all_tennessee_labels.csv')

# get rid of duplicate entries by just summing the amount of people in each row for rows w/ same tract
grouped_info = census_info.groupby('tract').sum()

df = grouped_info

# making percentage columns
df['white_pct'] = df['white alone'] / df['total population'] * 100
df['black_pct'] = df['black alone'] / df['total population'] * 100
df['native_pct'] = df['native alone'] / df['total population'] * 100
df['asian_pct'] = df['asian alone'] / df['total population'] * 100
df['hawaiin_pct'] = df['hawaiin alone'] / df['total population'] * 100
df['other_race_pct'] = df['other race alone'] / df['total population'] * 100
df['two_or_more_pct'] = df['two or more races'] / df['total population'] * 100
df['hispanic_pct'] = df['hispanic'] / df['total population'] * 100

# convert tract to int
joined['TRACTCE'] = joined['TRACTCE'].astype(int)

# group the original incidents dataset (which was joined with census tract data) by tract
grouped_joined = joined.groupby(["TRACTCE"]).agg({"response_time_sec": "mean",}).join(
    joined.groupby(["TRACTCE"])["response_time_sec"].count().rename("num_rows")
).reset_index().rename(
    columns={"response_time_sec": "avg_response_time_sec"}
)

# join the full original incidents with demographic info
joined2 = pd.merge(joined, df, how='left', left_on='TRACTCE', right_on='tract')

# join grouped incindents dataset with demograhpic info
joined3 = pd.merge(grouped_joined, df, how='left', left_on='TRACTCE', right_on='tract')

# rejoin the grouped demographic information with the geographic info for plotting
census_tracts['TRACTCE'] = census_tracts['TRACTCE'].astype(int) # convert the geographic info tractce

# print(joined3.head())
joined4 = pd.merge(joined3, census_tracts, how='left', left_on='TRACTCE', right_on='TRACTCE')

joined4.columns


<ipython-input-6-ac1ab0762110>:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped_info = census_info.groupby('tract').sum()


Index(['TRACTCE', 'avg_response_time_sec', 'num_rows', 'total population',
       'male 25 to 29', 'female 25 to 29', 'white alone', 'black alone',
       'native alone', 'asian alone', 'hawaiin alone', 'other race alone',
       'two or more races', 'hispanic', 'median household income',
       'median family income', 'total housing units', 'occupied housing units',
       'owner occupied housing units', 'housing units btw 500k and 750k',
       'housing units btw 750k and 1m', 'housing units above 1m',
       'vacant housing units', 'median housing value', 'median gross rent',
       'state', 'county', 'white_pct', 'black_pct', 'native_pct', 'asian_pct',
       'hawaiin_pct', 'other_race_pct', 'two_or_more_pct', 'hispanic_pct',
       'STATEFP', 'COUNTYFP', 'GEOID', 'NAME', 'NAMELSAD', 'MTFCC', 'FUNCSTAT',
       'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry'],
      dtype='object')